##### Copyright 2021 The TensorFlow Federated Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 학습을 위한 권장 집계 조정하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/tuning_recommended_aggregators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/federated/blob/v0.36.0/docs/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.36.0/docs/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/federated/docs/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

`tff.learning` 모듈에는 권장되는 기본 구성으로 모델 업데이트를 집계하는 여러 방법이 포함되어 있습니다.

- `tff.learning.robust_aggregator`
- `tff.learning.dp_aggregator`
- `tff.learning.compression_aggregator`
- `tff.learning.secure_aggregator`

이 튜토리얼에서는 기본적 동기와 이를 구현하는 방법을 설명하고 구성을 사용자 지정하는 방법을 제안합니다. 

---

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [1]:
import math
import tensorflow_federated as tff
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

집계 메서드는 `model_aggregator` 키워드 인수로 `tff.learning.algorithms.build_weighted_fed_avg`(및 `build_unweighted_fed_avg`)에 전달할 수 있는 객체로 표현됩니다. 따라서 여기에서 논의된 집계기는 페더레이션 학습에 대한 [이전](federated_learning_for_image_classification.ipynb) [튜토리얼](federated_learning_for_text_generation.ipynb)을 수정하는 데 직접 사용할 수 있습니다.

[FedAvg](http://proceedings.mlr.press/v54/mcmahan17a/mcmahan17a.pdf) 알고리즘의 기준선 가중 평균은 다음과 같이 `tff.aggregators.MeanFactory`를 사용하여 표현할 수 있습니다.

```
mean = tff.aggregators.MeanFactory()
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    ...,
    model_aggregator=mean)
```

이 튜토리얼에서 다루는 가중 평균을 확장하는 데 사용할 수 있는 기술은 다음과 같습니다.

- 제로화
- 클리핑
- 차등 프라이버시
- 압축
- 보안 집계

확장은 `MeanFactory`가 집계의 일부를 위임하는 내부 팩토리를 래핑하거나 다른 집계 팩토리에 의해 자체 래핑되는 합성을 사용하여 수행됩니다. 설계에 대한 자세한 내용은 [사용자 지정 집계기 구현](custom_aggregators.ipynb) 튜토리얼을 참조하세요.

먼저 이러한 기술을 개별적으로 활성화하고 구성하는 방법을 설명한 다음 함께 결합하는 방법을 보여드리겠습니다.

## 기술

개별 기술에 대해 자세히 알아보기 전에 먼저 아래 기술을 구성하는 데 유용한 분위수 일치 알고리즘을 소개합니다.

### 분위수 매칭

아래의 몇 가지 집계 기술은 집계의 일부 측면을 제어하는 기준 경계를 사용해야 합니다. 이러한 경계는 상수로 제공될 수 있지만 일반적으로 훈련 과정에서 경계를 조정하는 것이 좋습니다. 권장되는 방법은 [Andrew 등(2019)](https://arxiv.org/abs/1905.03871)의 분위수 매칭 알고리즘을 사용하는 것입니다. 처음에는 차등 프라이버시와의 호환성을 위해 제안되었지만 더 광범하게 활용되는 알고리즘입니다. 주어진 분위수에서 값을 추정하려면 `tff.aggregators.PrivateQuantileEstimationProcess`를 사용할 수 있습니다. 예를 들어 분포의 중앙값에 맞게 조정하려면 다음을 사용할 수 있습니다.

In [ ]:
median_estimate = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0, target_quantile=0.5, learning_rate=0.2)

앞으로 보게 되겠지만 분위수 추정 알고리즘을 사용하는 다른 기술에는 알고리즘 매개변수로 다른 값을 사용해야 합니다. 일반적으로 `learning_rate` 매개변수를 늘리면 올바른 분위수에 더 빨리 적응하지만 분산은 더 커집니다. `no_noise` classmethod는 차등 프라이버시를 위해 노이즈를 추가하지 않는 분위수 일치 프로세스를 구성합니다.

### 제로화

제로화는 비정상적으로 큰 값을 0으로 바꾸는 것을 말합니다. 여기서 "비정상적으로 크다"는 것은 미리 정의된 임계값보다 크거나 이전 계산 라운드의 값에 비해 크다는 것을 의미할 수 있습니다. 제로화는 결함이 있는 클라이언트의 데이터 손상에 대한 시스템 견고성을 높일 수 있습니다.

`ZEROING_CONSTANT`보다 큰 L-무한대 기준이 제로화되는 값의 평균을 계산하기 위해 `tff.aggregators.zeroing_factory`를 제로화를 수행하는 `tff.aggregators.MeanFactory`로 래핑합니다.

```
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=tff.aggregators.MeanFactory())
```

평균화를 통한 집계를 위해 내부 `MeanFactory`에 전달되기 전에 `zeroing_factory`의 (사전 집계) 효과가 클라이언트의 값에 적용되기를 원하므로 여기에서는 `MeanFactory`를 `zeroing_factory`로 래핑합니다.

그러나 대부분의 응용에서는 분위수 추정기를 사용한 적응형 제로화를 권장합니다. 이를 위해 다음과 같이 분위수 일치 알고리즘을 사용합니다.

In [ ]:
zeroing_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=10.0,
    target_quantile=0.98,
    learning_rate=math.log(10),
    multiplier=2.0,
    increment=1.0)
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=zeroing_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# zeroing_mean = tff.learning.robust_aggregator(clipping=False)

프로세스가 지금까지 본 가장 큰 값보다 다소 큰 값에 매우 빠르게(상대적으로 큰 `learning_rate`) 적응하도록 매개변수가 선택되었습니다. 분위수 추정치 `Q`의 경우, 제로화에 사용되는 임계값은 `Q * 승수 + 증분`입니다.

### 경계 L2 기준에 클리핑

클라이언트 업데이트를 클리핑(L2 공에 투영)하면 이상값에 대한 견고성을 향상시킬 수 있습니다. `tff.aggregators.clipping_factory`는 위에서 논의한 `tff.aggregators.zeroing_factory`와 정확히 같은 구조로 되어 있으며, 해당 `clipping_norm` 인수로 상수나 `tff.templates.EstimationProcess`를 취할 수 있습니다. 권장되는 모범 방식은 다음과 같이 적당히 높은 기준에 적당히 빠르게 적응하는 클리핑을 사용하는 것입니다.

In [ ]:
clipping_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0,
    target_quantile=0.8,
    learning_rate=0.2)
clipping_mean = tff.aggregators.clipping_factory(
    clipping_norm=clipping_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# clipping_mean = tff.learning.robust_aggregator(zeroing=False)

많은 문제에서 겪었던 경험에 따르면 학습률이 적절하게 조정되기만 한다면 `target_quantile`의 정확한 값은 그다지 중요하지 않은 것 같습니다. 그러나 매우 낮게 설정하면 클리핑을 사용하지 않는 것과 비교하여 최상의 성능을 얻기 위해 서버 학습률을 높여야 할 수 있으므로 기본적으로 0.8을 권장합니다.

### 차등 프라이버시

TFF는 적응형 클리핑 및 가우시안 노이즈를 사용하여 차등 비공개 집계도 지원합니다. 차등 비공개 평균화를 수행하는 집계기는 다음과 같이 구성할 수 있습니다.

In [ ]:
dp_mean = tff.aggregators.DifferentiallyPrivateFactory.gaussian_adaptive(
    noise_multiplier=0.1, clients_per_round=100)

# Equivalent to:
# dp_mean = tff.learning.dp_aggregator(
#   noise_multiplier=0.1, clients_per_round=100, zeroing=False)

`noise_multiplier` 인수를 설정하는 방법에 대한 지침은 [TFF DP 튜토리얼](https://www.tensorflow.org/federated/tutorials/federated_learning_with_differential_privacy)에서 찾을 수 있습니다.

### 손실 압축

gzip과 같은 무손실 압축과 비교할 때 손실 압축은 일반적으로 압축률을 훨씬 더 높으며, 이후에 여전히 무손실 압축과 결합시킬 수 있습니다. 클라이언트-서버 통신에 필요한 시간이 줄어들기 때문에 훈련 라운드가 더 빨리 완료됩니다. 학습 알고리즘의 본질적으로 무작위화된 특성으로 인해 어느 정도의 임계값까지는 손실 압축으로 인한 부정확성이 전체 성능에 부정적인 영향을 미치지 않습니다.

기본적인 권장은 텐서 크기 압축 <code>threshold</code>과 `quantization_bits` 수의 두 값으로 매개변수화된 단순 균일 양자화(예: [Suresh 등](http://proceedings.mlr.press/v70/suresh17a/suresh17a.pdf) 참조)를 사용하는 것입니다. 모든 텐서 `t`에 대해 `t`의 요소 수가 `threshold`보다 작거나 같으면 압축되지 않습니다. 더 크면 `t`의 요소가 `quantizaton_bits` 비트로의 무작위 반올림을 사용하여 양자화됩니다. 즉, 우리는 다음 연산을 적용하여

`t = round((t - min(t)) / (max(t) - min(t)) * (2**quantizaton_bits - 1)),`

`[0, 2**quantizaton_bits-1]` 범위의 정수 값을 생성합니다. 양자화된 값을 정수형으로 직접 패킹하여 전송한 후 역변환을 적용합니다.

`quantizaton_bits`를 8로, `threshold`를 20000으로 설정하는 것이 좋습니다.


In [ ]:
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))

# Equivalent to:
# compressed_mean = tff.learning.compression_aggregator(zeroing=False, clipping=False)

#### 조정 제안

두 매개변수, `quantization_bits` 및 `threshold`를 조정할 수 있으며 각 훈련 라운드에 참여하는 클라이언트의 수도 압축의 효율성에 영향을 줄 수 있습니다.

**임계값.** 기본 값 20000이 선택되었습니다. 그 이유는 공통 레이어 유형의 편향과 같이 요소 수가 적은 변수가 도입된 노이즈에 훨씬 더 민감하다는 것을 관찰했기 때문입니다. 게다가, 압축되지 않은 크기가 처음에는 상대적으로 작기 때문에 실제로 적은 수의 요소로 변수를 압축하여 얻을 수 있는 이득이 거의 없습니다.

일부 응용에서는 임계값 선택을 변경하는 것이 합리적일 수 있습니다. 예를 들어, 분류 모델의 출력 레이어에 나타나는 편향은 노이즈에 더 민감할 수 있습니다. 어휘가 20004인 언어 모델을 훈련하는 경우라면 `threshold`를 20004로 설정해야 할 수 있습니다.

**양자화 비트.** `quantization_bits`의 기본값 8은 대부분의 사용자에게 적합합니다. 8이 잘 작동하고 성능을 조금 더 끌어내고 싶다면 7이나 6으로 낮춰서 시도해 보세요. 작게 그리드 검색을 수행할 수 있을 정도의 여력이 된다면 훈련이 불안정해지거나 최종 모델의 품질이 저하되기 시작하는 값을 확인한 다음 해당 값을 2만큼 늘리는 방법을 추천합니다. 예를 들어, `quantization_bits`를 5로 설정하면 잘 작동하지만 4로 설정하면 모델 품질이 저하되는 경우 "안전"을 위해 기본값 6을 권장합니다.

**라운드당 클라이언트 수.** 라운드당 클라이언트 수를 크게 늘리면 더 작은 `quantization_bits` 값에서 잘 작동할 수 있습니다. 양자화에 의해 도입된 무작위 부정확성이 더 많은 클라이언트 업데이트에 걸쳐 평균화되어 상쇄될 수 있기 때문입니다.

### 보안 집계

보안 집계(SecAgg)란 서버가 합계만 해독할 수 있는 방식으로 클라이언트 업데이트가 암호화되는 암호화 프로토콜을 말합니다. 다시 보고하는 클라이언트의 수가 충분하지 않은 경우 서버는 전혀 학습하지 않으며 어떤 경우에도 서버가 개별 업데이트를 검사할 수 없습니다. 이를 실현하기 위해 `tff.federated_secure_sum_bitwidth` 연산자를 사용할 수 있습니다.

모델 업데이트는 부동 소수점 값이지만 SecAgg는 정수에서 작동합니다. 따라서 정수 형식으로 이산화하기 전에 큰 값을 일부 경계로 잘라야 합니다. 클리핑 경계는 상수이거나 조절하는 방식(이것이 권장됨)으로 결정될 수 있습니다. 그런 다음 정수가 안전하게 합산되고 합계가 부동 소수점 도메인으로 다시 매핑됩니다.

`MY_SECAGG_BOUND`를 클리핑 경계로 사용하여 SecAgg로 합산된 가중치 값으로 평균을 계산하려면 `SecureSumFactory`를 `MeanFactory`에 다음과 같이 전달합니다.

```
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(MY_SECAGG_BOUND))
```

경계를 조절 방식으로 결정하면서 동일한 작업을 수행하려면:

In [ ]:
secagg_bound = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=50.0,
    target_quantile=0.95,
    learning_rate=1.0,
    multiplier=2.0)
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(secagg_bound))

# Equivalent to:
# secure_mean = tff.learning.secure_aggregator(zeroing=Fasle, clipping=False)

#### 조정 제안

경계에 여유가 없지만(이산화에서 많은 정밀도를 잃지 않을 것입니다) 클리핑이 거의 발생하지 않도록 조정 매개변수가 선택되었습니다.

매개변수를 조정하는 경우, SecAgg 프로토콜은 평균에 가중치를 부여한 후 가중치가 적용된 모델 업데이트를 합산합니다. 가중치는 일반적으로 로컬에서 처리되는 데이터 포인트의 수이므로 서로 다른 작업 간에 올바른 경계는 이 데이터의 양에 따라 달라질 수 있습니다.

적응형 `secagg_bound`를 생성할 때 `increment` 키워드 인수를 사용하지 않는 것이 좋습니다. 그렇지 않으면 실제 추정치가 작아지는 경우 상대적 정밀도 손실이 커질 수 있기 때문입니다.

위의 코드 조각은 가중치가 적용된 값만 SecAgg를 사용합니다. 가중치 합계에도 SecAgg를 사용해야 하는 경우, 일반적인 훈련의 경우와 같이 경계를 상수로 설정하는 것이 좋습니다. 가능한 가장 큰 가중치를 미리 알 수 있습니다.

```
secure_mean = tff.aggregators.MeanFactory(
    value_sum_factory=tff.aggregators.SecureSumFactory(secagg_bound),
    weight_sum_factory=tff.aggregators.SecureSumFactory(
        upper_bound_threshold=MAX_WEIGHT, lower_bound_threshold=0.0))
```

## 구성 기술

위에 소개된 평균을 확장하는 개별 기술을 함께 결합할 수 있습니다.

이러한 기술을 클라이언트에 다음 순서로 적용할 것을 권장합니다.

1. 제로화
2. 클리핑
3. 기타 기술

`tff.aggregators` 모듈의 집계기는 "외부 집계기" 안에 "내부 집계기"(이것의 사전 집계 효과가 마지막에 발생하고 사후 집계 효과가 먼저 발생)를 래핑하여 구성됩니다. 예를 들어 제로화, 클리핑 및 압축(이 순서로)을 수행하려면 다음과 같이 작성합니다.

```
# Compression is innermost because its pre-aggregation effects are last.
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))
# Compressed mean is inner aggregator to clipping...
clipped_compressed_mean = tff.aggregators.clipping_factory(
    clipping_norm=MY_CLIPPING_CONSTANT,
    inner_agg_factory=compressed_mean)
# ...which is inner aggregator to zeroing, since zeroing happens first.
final_aggregator = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=clipped_compressed_mean)
```

이 구조는 학습 알고리즘의 [기본 집계기](https://github.com/tensorflow/federated/blob/11e4f632b38745c9b38cc39fa1fe67771c206e77/tensorflow_federated/python/learning/model_update_aggregator.py)와 일치합니다.

다른 구성도 가능합니다. 다른 여러 응용에서도 작동하는 기본 구성을 제공할 수 있다고 확신하게 되면 이 문서를 확장할 것입니다. 새로운 아이디어를 구현하려면 [사용자 지정 집계기 구현](custom_aggregators.ipynb) 튜토리얼을 참조하세요.